In [7]:
from datasets import load_dataset

reviews_test = load_dataset("imdb", split='test')
reviews_train = load_dataset("imdb", split='train')

Reusing dataset imdb (/home/meirav-gri/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset imdb (/home/meirav-gri/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


In [8]:
print(reviews_test[0])
reviews_train[0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
reviews_test_vector = [ model.encode(review) for review in reviews_test[:20]['text']]
#reviews_train_vector = [ model.encode(review) for review in reviews_train[:]['text']]
#%store reviews_train_vector

"""
    load embedded reviews_train_vector
    if this returns an error, uncomment 2 previous lines
"""
%store -r reviews_train_vector

#check how many dimensions in a single vector 
reviews_test_vector[0].shape
reviews_train_vector[0].shape

vector_dim = reviews_test_vector[0].shape

In [10]:
from scipy.spatial.distance import euclidean, cosine

def general_distance_lookup(trained_data, trained_vectors, query_data, query_vector, requested_k, dist_metric):
    #for each trained_vector
    distances = list()
    for idx, trained_vector in enumerate(trained_vectors):
        
        #calc distance to test_vector
        dist = dist_metric(trained_vector, query_vector)
        
        
        #append distance to dists_list
        distances.append((idx, dist))
        
    #sort distances from lowest to highest
    distances.sort(key=lambda tup: tup[1])
    
    print(f"""we looked for: {query_data}
            
            we found:""")
    
    #print text of requested_k minimum of dists_list
    for result in range(requested_k):
        
        train_index = distances[result][0]
        train_review = trained_data[train_index]
        print ("********REVIEW NUMBER: " + str(train_index) + ' ********')
        print(f"{train_review}\n")

In [11]:
 general_distance_lookup(reviews_train, reviews_train_vector,reviews_test[0], reviews_test_vector[0], 5, euclidean)

we looked for: {'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it

In [12]:
general_distance_lookup(reviews_train, reviews_train_vector,reviews_test[0], reviews_test_vector[0], 5, cosine)

we looked for: {'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it

In [14]:
import redis

#connect to redis
host = 'vecsim'
port = 6379
redis_conn = redis.Redis(host = host, port = port)
print ('Connected to redis')

Connected to redis


In [15]:
#load reviews into redis hash
import numpy as np
def load_vectors(client:redis.Redis, trained_data, data_vectors, vector_field_name):
    #map all trained data
    p = client.pipeline(transaction=False)
    reviews_train[0]['label']
    for index, entry in enumerate(trained_data):
        
        #hash key
        key = 'review:'+ str(index)
        
        #hash fields
        text = entry['text']
        label = entry['label']  
        data_vector = data_vectors[index].astype(np.float32).tobytes()
        data_mapping = {'text':text,'label':label, vector_field_name:data_vector}
        
        p.hset(key,mapping = data_mapping) 
    p.execute()

In [16]:
#Utility Functions to Create Indexes on Vector field

def create_flat_index (redis_conn,vector_field_name,number_of_vectors, vector_dimensions = vector_dim, distance_metric='COSINE'):
    create_command =  ["FT.CREATE", "idx", "SCHEMA","text","TEXT","label","TAG"]
    create_command += ["reviews_vector", "VECTOR", "FLAT", "6", 
                        "TYPE", "FLOAT32", 
                        "DIM", str(vector_dimensions), 
                        "DISTANCE_METRIC", str(distance_metric)]
    redis_conn.execute_command(*create_command)

def create_hnsw_index (redis_conn, vector_dimensions = vector_dim, distance_metric='COSINE'):
    
    create_command =  ["FT.CREATE", "idx", "SCHEMA","text","TEXT","label","TAG"]
    create_command += ["reviews_train_vector", "VECTOR", "HNSW", "6", 
                        "TYPE", "FLOAT32", 
                        "DIM", str(vector_dimensions), 
                        "DISTANCE_METRIC", str(distance_metric)]
    
    redis_conn.execute_command(*create_command)

In [17]:
VECTOR_FIELD_NAME = 'review_vector'
DISTANCE_METRIC = 'COSINE'
DIMENSIONS = vector_dim

redis_conn.flushall()
#create_hnsw_index(redis_conn,DIMENSIONS,DISTANCE_METRIC)
#load_vectors(redis_conn,reviews_train,reviews_test_vector,VECTOR_FIELD_NAME)
#print(type(headline_vectors[0]))

ConnectionError: Error -2 connecting to vecsim:6379. Name or service not known.